# Basic Monochrome Imager on a Drone

Computing the basic parameters of a monochrome imager on a drone. Valid for a single band.

Reference parameters for the scene and camera position with respect to the target are given below.

In [77]:
from pint import UnitRegistry
import numpy as np

# Init units
u = UnitRegistry()

# positional params
# -----------------
distance = 10. * u.km

# scene ref params
# ----------------

# blue: 450-485 nm
# green: 500-565 nm
# red: 625-750 nm
# nir: 780-2500 nm
ref_wavelength = 632 * u.nm


Optical Parameters are given below.

In [78]:
# optical params (design)
# -----------------------

focal_length = 1270 * u.mm
aperture_diam = (25 * u.inch).to("mm")
# image diameter on the focal plane (fixed by the design)
# image_diam_on_focal_plane = 25 * u.mm 
image_diam_on_focal_plane = (np.sqrt(1920**2 + 1080**2) * 12 * u.um).to("mm")
# image_diam_on_focal_plane = (np.sqrt(1080**2 + 720**2)  * 12 * u.um).to("mm")

# optical params (computed)
# -------------------------

f_nr = focal_length / aperture_diam
full_fov = 2 * np.arctan((image_diam_on_focal_plane/2.) / focal_length)
aperture_area = np.pi * (aperture_diam/2.)**2
# solid angle = 2pi h/r
aperture_solid_angle = np.pi / (focal_length / (aperture_diam/2.))**2 * u.steradian

u.define('lp = 1 * dimensionless = lp')
spatial_cutoff_freq = (1. * u.lp/(ref_wavelength*f_nr).to(u.mm)) # perfect incoherent optics

print(f"F-number : {f_nr:~.4}")
print(f"Full FoV : {full_fov.to(u.deg):~.4}")
print(f"image_diam_on_focal_plane : {image_diam_on_focal_plane:~.4}")

print(f"Spatial cut-off frequency : {spatial_cutoff_freq:~.4P}")

F-number : 2.0
Full FoV : 1.193 deg
image_diam_on_focal_plane : 26.43 mm
Spatial cut-off frequency : 791.1 lp/mm


Detector Parameters are given here

In [79]:
# Detector Params (design)
# ------------------------

pix_pitch = 12 * u.um
horiz_pixels: int = 1920 
horiz_pixels_used: int = 1920
vert_pixels: int = 1080
vert_pixels_used: int = 1080

binning: int = 1 # should be int

# frame duration that cannot be used for inetgration or imaging (DUMMY VALUES)
frame_overhead_time = 35 * u.us 
# time between frame durations that can be used for inetgration or imaging (DUMMY VALUES)
overlap_time = 15 * u.us

# frame rate
frame_rate = 29.97 * u.Hz


# Detector Params (computed)
# --------------------------

nyq_freq_native = 1 * u.lp/(2*pix_pitch).to(u.mm)
nyq_freq = 1 * u.lp/(2*binning*pix_pitch).to(u.mm)

if binning==1:  
    print(f"Nyquist Frequency : {nyq_freq_native:~.4P}")
else:
    print(f"Nyquist Frequency (native) : {nyq_freq_native:~.4P}")
    print(f"Nyquist Frequency (binned) : {nyq_freq:~.4P}")

frame_pixel_count_used = horiz_pixels_used * vert_pixels_used * u.pixel
print(f"Number of pixels (used) : {frame_pixel_count_used.to("Mpixel"):~.4}")

# Instantaneous Field of View (works in vertical and horizontal)
ifov_native = 2*np.arctan( (pix_pitch / 2.) /focal_length)
ifov = 2*np.arctan( (pix_pitch * binning / 2.) /focal_length)

if binning==1:  
    print(f"IFoV : {ifov_native.to(u.mdeg):~.4P}  ({ifov_native.to(u.urad):~.4P})")
else:
    print(f"IFoV (native) : {ifov_native.to(u.mdeg):~.4P}  ({ifov_native.to(u.urad):~.4P})")
    print(f"IFoV (binned) : {ifov.to(u.mdeg):~.4P}  ({ifov.to(u.urad):~.4P})")

# pixel physical area
pix_area = pix_pitch**2

# Pixel solid angle (of a pyramid)
pix_solid_angle_native = 4 * np.arcsin( np.sin(ifov_native/2.0) * np.sin(ifov_native/2.0) ) 
pix_solid_angle_native = (pix_solid_angle_native * u.rad).to(u.steradian) # correct the unit from rad to sr

pix_solid_angle = 4 * np.arcsin( np.sin(ifov/2.0) * np.sin(ifov/2.0) ) 
pix_solid_angle = (pix_solid_angle * u.rad).to(u.steradian) # correct the unit from rad to sr

if binning==1: 
    print(f"Pixel solid angle : {pix_solid_angle_native:~.4P}")
else:
    print(f"Pixel solid angle (native) : {pix_solid_angle_native:~.4P}")
    print(f"Pixel solid angle (binned) : {pix_solid_angle:~.4P}")

# Full FoVs
horiz_fov = 2*np.tan(ifov_native * horiz_pixels_used /2.)
vertical_fov = 2*np.tan(ifov_native * vert_pixels_used /2.)

print(f"Horizontal Full FoV : {horiz_fov.to(u.deg):~.4P} (used pixels only)")
print(f"Vertical Full FoV   : {vertical_fov.to(u.deg):~.4P} (used pixels only)")


Nyquist Frequency : 41.67 lp/mm
Number of pixels (used) : 2.074 Mpixel
IFoV : 0.5414 mdeg  (9.449 µrad)
Pixel solid angle : 8.928×10⁻¹¹ sr
Horizontal Full FoV : 1.039 deg (used pixels only)
Vertical Full FoV   : 0.5847 deg (used pixels only)


Geometric Output Parameters

(Fwd Motion Compensation not implemented)

In [80]:
# Geometric Output Parameters (computed)
# --------------------------------------

print(f"Target Distance : {distance.to(u.km):~.4P}") 

# Ground sample distance at nadir
spatial_sample_distance_native = (distance*(pix_pitch/focal_length)).to_reduced_units().to(u.m)
spatial_sample_distance = (distance*(pix_pitch * binning/focal_length)).to_reduced_units().to(u.m)

# image width at target distance flat plate and constant Instantaneous FoV
image_width_at_dist = 2*np.tan(ifov_native * horiz_pixels_used /2.) * distance
image_height_at_dist = 2*np.tan(ifov_native * vert_pixels_used /2.) * distance

if binning==1: 
    print(f"Spatial Sample Distance : {spatial_sample_distance_native.to(u.cm):~.4P}  (@ target distance)")
else:
    print(f"Spatial Sample Distance (native) : {spatial_sample_distance_native.to(u.cm):~.4P}  (@ target distance)")
    print(f"Spatial Sample Distance (binned) : {spatial_sample_distance.to(u.cm):~.4P}  (@ target distance)")

print(f"Image Width  : {image_width_at_dist.to(u.km):~.4P}  (@ target distance)") 
print(f"Image Height : {image_height_at_dist.to(u.km):~.4P}  (@ target distance)") 


Target Distance : 10.0 km
Spatial Sample Distance : 9.449 cm  (@ target distance)
Image Width  : 0.1814 km  (@ target distance)
Image Height : 0.102 km  (@ target distance)


Timings

Note the user defined value integration duration (though limited by the computed max integration duration). If the integration time is longer, more photons are captured but image smear increases.

In [81]:
# Timings
# -------

# native line max integration duration possible
max_integ_duration = (1/frame_rate) - frame_overhead_time + overlap_time

# acutal integration duration (<= max_integ_duration) set by the user, in this example as a percentage
integ_duration = max_integ_duration * 0.4 

print(f"Max integ duration : {max_integ_duration.to(u.ms):~.4P}")
print(f"Actual integ duration : {integ_duration.to(u.ms):~.4P} ({(integ_duration/max_integ_duration).m:.1%} of max)")


Max integ duration : 33.35 ms
Actual integ duration : 13.34 ms (40.0% of max)


Readout electronics

In [86]:
# Read-out Electronics Params (design)
# ------------------------------------

pixel_encoding = 12 * u.bpp

# print(f"Pixel Encoding : {pixel_encoding}")

# data overhead during readout and write
data_write_overhead = 2./100.

# compression
compression_on = False 
compression_ratio = 2.4


# Read-out Electronics Params (computed)
# --------------------------------------

print(f"Read rate {'' if binning==1 else '(native)'}: {(horiz_pixels * vert_pixels * u.pixel * frame_rate).to('Mpixel/s'):~.5P}")

pixel_read_datarate_native= horiz_pixels * vert_pixels * u.pixel * pixel_encoding * frame_rate
pixel_read_datarate = (horiz_pixels / binning) * (vert_pixels / binning) * u.pixel * pixel_encoding * frame_rate # with binning

if binning==1: 
    print(f"Read datarate : {pixel_read_datarate_native.to('Mbit/s'):~.4P}  ({pixel_read_datarate_native.to('Mbyte/s'):~.4P})")
else:
    print(f"Read datarate (native) : {pixel_read_datarate_native.to('Mbit/s'):~.4P}  ({pixel_read_datarate_native.to('Mbyte/s'):~.4P})")
    print(f"Read datarate (binned) : {pixel_read_datarate.to('Mbit/s'):~.4P}  ({pixel_read_datarate.to('Mbyte/s'):~.4P})")

# init write datarate (with data compression if applicable)
if compression_on:
    pixel_write_datarate_native = pixel_read_datarate_native / compression_ratio
    pixel_write_datarate = pixel_read_datarate / compression_ratio
else:
    pixel_write_datarate_native = pixel_read_datarate_native
    pixel_write_datarate = pixel_read_datarate 

pixel_write_datarate_native = pixel_write_datarate_native * (1+data_write_overhead)
pixel_write_datarate = pixel_write_datarate * (1.+data_write_overhead)

if binning==1: 
    print(f"Write datarate : {pixel_write_datarate_native.to('Mbit/s'):~.4P}  ({pixel_write_datarate_native.to('Mbyte/s'):~.4P})  (after overheads{' and compression' if compression_on else ''})")
else:
    print(f"Write datarate (native) : {pixel_write_datarate_native.to('Mbit/s'):~.4P}  ({pixel_write_datarate_native.to('Mbyte/s'):~.4P})  (after overheads{' and compression' if compression_on else ''}")
    print(f"Write datarate (binned) : {pixel_write_datarate.to('Mbit/s'):~.4P}  ({pixel_write_datarate.to('Mbyte/s'):~.4P})  (after overheads{' and compression' if compression_on else ''}")


Read rate : 62.146 Mpixel/s
Read datarate : 745.7 Mbit/s  (93.22 MB/s)
Write datarate : 760.7 Mbit/s  (95.08 MB/s)  (after overheads)
